In [ ]:
from win32com.client.gencache import EnsureDispatch, EnsureModule
from win32com.client import CastTo, constants
import time

# Notes
#
# The python project and script was tested with the following tools:
#       Python 3.4.3 for Windows (32-bit) (https://www.python.org/downloads/) - Python interpreter
#       Python for Windows Extensions (32-bit, Python 3.4) (http://sourceforge.net/projects/pywin32/) - for COM support
#       Microsoft Visual Studio Express 2013 for Windows Desktop (https://www.visualstudio.com/en-us/products/visual-studio-express-vs.aspx) - easy-to-use IDE
#       Python Tools for Visual Studio (https://pytools.codeplex.com/) - integration into Visual Studio
#
# Note that Visual Studio and Python Tools make development easier, however this python script should should run without either installed.

class PythonStandaloneApplication(object):
    class LicenseException(Exception):
        pass

    class ConnectionException(Exception):
        pass

    class InitializationException(Exception):
        pass

    class SystemNotPresentException(Exception):
        pass

    def __init__(self):
        # make sure the Python wrappers are available for the COM client and
        # interfaces
        EnsureModule('ZOSAPI_Interfaces', 0, 1, 0)
        # Note - the above can also be accomplished using 'makepy.py' in the
        # following directory:
        #      {PythonEnv}\Lib\site-packages\wind32com\client\
        # Also note that the generate wrappers do not get refreshed when the
        # COM library changes.
        # To refresh the wrappers, you can manually delete everything in the
        # cache directory:
        #	   {PythonEnv}\Lib\site-packages\win32com\gen_py\*.*
        
        self.TheConnection = EnsureDispatch("ZOSAPI.ZOSAPI_Connection")
        if self.TheConnection is None:
            raise PythonStandaloneApplication.ConnectionException("Unable to intialize COM connection to ZOSAPI")

        self.TheApplication = self.TheConnection.CreateNewApplication()
        if self.TheApplication is None:
            raise PythonStandaloneApplication.InitializationException("Unable to acquire ZOSAPI application")

        if self.TheApplication.IsValidLicenseForAPI == False:
            raise PythonStandaloneApplication.LicenseException("License is not valid for ZOSAPI use")

        self.TheSystem = self.TheApplication.PrimarySystem
        if self.TheSystem is None:
            raise PythonStandaloneApplication.SystemNotPresentException("Unable to acquire Primary system")

    def __del__(self):
        if self.TheApplication is not None:
            self.TheApplication.CloseApplication()
            self.TheApplication = None

        self.TheConnection = None

    def OpenFile(self, filepath, saveIfNeeded):
        if self.TheSystem is None:
            raise PythonStandaloneApplication.SystemNotPresentException("Unable to acquire Primary system")
        self.TheSystem.LoadFile(filepath, saveIfNeeded)

    def CloseFile(self, save):
        if self.TheSystem is None:
            raise PythonStandaloneApplication.SystemNotPresentException("Unable to acquire Primary system")
        self.TheSystem.Close(save)

    def SamplesDir(self):
        if self.TheApplication is None:
            raise PythonStandaloneApplication.InitializationException("Unable to acquire ZOSAPI application")

        return self.TheApplication.SamplesDir

    def ExampleConstants(self):
        if self.TheApplication.LicenseStatus is constants.LicenseStatusType_PremiumEdition:
            return "Premium"
        elif self.TheApplication.LicenseStatus is constants.LicenseStatusType_ProfessionalEdition:
            return "Professional"
        elif self.TheApplication.LicenseStatus is constants.LicenseStatusType_StandardEdition:
            return "Standard"
        else:
            return "Invalid"


if __name__ == '__main__':
    zosapi = PythonStandaloneApplication()
    value = zosapi.ExampleConstants()

    # written with ZOSAPI 16.5, 20161019, MRH

    # Set up primary optical system
    TheSystem = zosapi.TheSystem
    TheApplication = zosapi.TheApplication
    sampleDir = TheApplication.SamplesDir
    # Open file
    testFile = sampleDir + '\\Sequential\\Objectives\\Single Lens Example - Python.zmx'
    TheSystem.LoadFile(testFile, False)
    testFile2 = sampleDir + '\\Sequential\Objectives\\Single Lens Example optimised - Python.zmx'
    TheSystem.SaveAs(testFile2)

    # Get Surfaces
    TheLDE = TheSystem.LDE
    Surface_1 = TheLDE.GetSurfaceAt(1)
    Surface_2 = TheLDE.GetSurfaceAt(2)
    Surface_3 = TheLDE.GetSurfaceAt(3)

    #Save and close
    TheSystem.Save()

    # This will clean up the connection to OpticStudio.
    # Note that it closes down the server instance of OpticStudio, so you for maximum performance do not do
    # this until you need to.
    del zosapi
    zosapi = None





In [1]:
# 1. physical optics in LDE
s1 = TheLDE.GetSurfaceAt(1)
s1_pop = s1.PhysicalOpticsData
s1_pop.ResampleAfterRefraction = True
s1_pop.AutoResample = True

# 2. Change to Slide Surface & update slide image
TheLDE.InsertNewSurfaceAt(2)
s2 = TheLDE.GetSurfaceAt(2)
s2_type = s2.GetSurfaceTypeSettings(constants.SurfaceType_SlideSurface)
s2_type.fileName = 'barchart.BMP'
s2.ChangeType(s2_type)

# 2.5 Modify Settings Method
# Modify settings method
pop = TheSystem.Analyses.New_Analysis_SettingsFirst(constants.AnalysisIDM_PhysicalOpticsPropagation)
pop.Terminate()
pop_setting = pop.GetSettings()
pop_settings = CastTo(pop_setting, 'IAS_')

if pop.HasAnalysisSpecificSettings == False:
    cfg = TheApplication.ZemaxDataDir + '\\Configs\\POP_IRRADIANCE.CFG'
    pop_settings.ModifySettings(cfg, 'POP_END', (TheLDE.NumberOfSurfaces - 1))

    # changes beam to Guassian, 128x128, Auto, 10x15um waist
    pop_settings.ModifySettings(cfg, 'POP_BEAMTYPE', 0)
    pop_settings.ModifySettings(cfg, 'POP_SAMPX',3)
    pop_settings.ModifySettings(cfg, 'POP_SAMPY',3)
    pop_settings.ModifySettings(cfg, 'POP_PARAM1', 0.010)    # Waist X
    pop_settings.ModifySettings(cfg, 'POP_PARAM2', 0.015)    # Waist Y
    pop_settings.ModifySettings(cfg, 'POP_AUTO', 1)          # needs to go after POP_PARAM
    
    pop_settings.LoadFrom(cfg)


# 3. Run POP and save output
pop.ApplyAndWaitForCompletion()
pop_results = pop.GetResults()
pop_results.GetTextFile('C:\\Temp\\pop_irradiance.txt')

# --------------------------------------------------------------------------------------------
# this is a repeat of 2.5 and 3; it was easier to simply copy & paste and change POP_PHASE.CFG file
pop2 = TheSystem.Analyses.New_Analysis_SettingsFirst(constants.AnalysisIDM_PhysicalOpticsPropagation)
pop2.Terminate()
pop_setting = pop2.GetSettings()
pop_settings = CastTo(pop_setting, 'IAS_')

cfg = TheApplication.ZemaxDataDir + '\\Configs\\POP_PHASE.CFG'
pop_settings.ModifySettings(cfg, 'POP_END', (TheLDE.NumberOfSurfaces - 1))
pop_settings.ModifySettings(cfg, 'POP_BEAMTYPE', 0)
pop_settings.ModifySettings(cfg, 'POP_SAMPX', 3)
pop_settings.ModifySettings(cfg, 'POP_SAMPY', 3)
pop_settings.ModifySettings(cfg, 'POP_PARAM1', 0.010)  # Waist X
pop_settings.ModifySettings(cfg, 'POP_PARAM2', 0.015)  # Waist Y
pop_settings.ModifySettings(cfg, 'POP_AUTO', 1)  # needs to go after POP_PARAM
pop_settings.LoadFrom(cfg)

pop2.ApplyAndWaitForCompletion()
pop_results = pop2.GetResults()
pop_results.GetTextFile('C:\\Temp\\pop_phase.txt')

# --------------------------------------------------------------------------------------------

# 4. System Aperture Float By Stop Size
TheSystem.SystemData.Aperture.ApertureType = 3

NameError: name 'TheLDE' is not defined